In [ ]:
# --- Markdown ---
# ## 1. Setup and Data Loading
#
# Import necessary libraries and load data from CSV files.
# Prepare data structures for both the distance calculation function and the Pyomo model.
# --- End Markdown ---

# --- Python Code ---
%pip install pyomo
import pyomo.environ as pyo
import pandas as pd
import requests   # To make HTTP requests to OSRM API
import json       # To parse JSON responses
import time       # To potentially add delays for rate limiting
import sys        # For error messages
import math       # For fallback distance calc if needed

# --- Configuration ---
# OSRM Server URL (Using the public demo server)
OSRM_BASE_URL = "http://router.project-osrm.org"

# Define the cost per kilometer (as per notebook)
C_KM = 20700 # COP/km

# --- Load Data from CSV ---
try:
    depots_df = pd.read_csv("case_3_base/Depots.csv")
    clients_df = pd.read_csv("case_3_base/Clients.csv")
    vehicles_df = pd.read_csv("case_3_base/Vehicles.csv")
    print("CSV files loaded successfully.")
except FileNotFoundError as e:
    print(f"ERROR: Could not find CSV file: {e}. Make sure the files are in the same directory.", file=sys.stderr)
    sys.exit(1) # Exit if data files are missing
except Exception as e:
    print(f"ERROR: An error occurred while loading CSV files: {e}", file=sys.stderr)
    sys.exit(1)

# --- Process Data for Pyomo & the compute_distance_matrix function ---

# A. Pyomo-specific Sets
set_I_data = [f'D{depot_id}' for depot_id in depots_df['DepotID']]
set_J_data = [f'C{client_id}' for client_id in clients_df['ClientID']]
set_K_data = [f'V{i+1}' for i in range(len(vehicles_df))] # Vehicle IDs V1, V2, ...
set_N_data = set_I_data + set_J_data

# B. Pyomo-specific Parameters
param_A_data = {f'D{depots_df.loc[i, "DepotID"]}': float('inf') for i in depots_df.index}
print("WARNING: Depot capacities (A_i) not found in Depots.csv. Assuming infinite capacity.")

param_D_data = {f'C{clients_df.loc[i, "ClientID"]}': clients_df.loc[i, 'Product']
                for i in clients_df.index}
param_Q_data = {f'V{i+1}': vehicles_df.loc[i, 'Capacity'] for i in vehicles_df.index}
param_R_data = {f'V{i+1}': vehicles_df.loc[i, 'Range'] for i in vehicles_df.index}
param_n_cust_data = len(set_J_data)

# C. Data Dictionary for compute_distance_matrix function
data = {}
data['N'] = set_N_data # Set of Node IDs
data['UbicacionNodo'] = {} # Node coordinates {NodeID: (lat, lon)}

data['ID'] = list(vehicles_df['VehicleID'].unique()) # List of unique vehicle types
#print (f"Vehicle IDs: {data['ID']}")

# Populate coordinates dictionary (lat, lon)
for i in depots_df.index:
    node_id = f'D{depots_df.loc[i, "DepotID"]}'
    data['UbicacionNodo'][node_id] = (depots_df.loc[i, 'Latitude'], depots_df.loc[i, 'Longitude'])
for i in clients_df.index:
    node_id = f'C{clients_df.loc[i, "ClientID"]}'
    data['UbicacionNodo'][node_id] = (clients_df.loc[i, 'Latitude'], clients_df.loc[i, 'Longitude'])

# Initialize the nested distance dictionary expected by the function
data['d'] = {tv: {u: {} for u in data['N']} for tv in data['ID']}

# D. Placeholder Parameters for Pyomo (will be filled by OSRM results)
param_d_data = {} # Will store {(u, v): distance_km}
param_c_data = {} # Will store {(u, v): cost_cop}



print("--- Data Prepared for Pyomo and Distance Function ---")
print(f"Set I (Depots): {set_I_data}")
print(f"Set J (Customers): {set_J_data}")
print(f"Set K (Vehicles): {set_K_data}")
print(f"Set N (Nodes): {data['N']}")
print(f"Param D (Demands): {param_D_data}")
print(f"Param Q (Veh. Caps): {param_Q_data}")
print(f"Param R (Veh. Ranges): {param_R_data}") 
print(f"Param n_cust: {param_n_cust_data}")
# print(f"Vehicle Types (CV): {data['CV']}")
print(f"Node Locations (UbicacionNodo): {data['UbicacionNodo']}") # Can be long
print("--- End Data Preparation ---")
# --- End Python Code ---

Note: you may need to restart the kernel to use updated packages.
CSV files loaded successfully.
{'N': ['D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C23', 'C24', 'C25', 'C26', 'C27', 'C28', 'C29', 'C30', 'C31', 'C32', 'C33', 'C34', 'C35', 'C36', 'C37', 'C38', 'C39', 'C40', 'C41', 'C42', 'C43', 'C44', 'C45', 'C46', 'C47', 'C48', 'C49', 'C50', 'C51', 'C52', 'C53', 'C54', 'C55', 'C56', 'C57', 'C58', 'C59', 'C60', 'C61', 'C62', 'C63', 'C64', 'C65', 'C66', 'C67', 'C68', 'C69', 'C70', 'C71', 'C72', 'C73', 'C74', 'C75', 'C76', 'C77', 'C78', 'C79', 'C80', 'C81', 'C82', 'C83', 'C84', 'C85', 'C86', 'C87', 'C88', 'C89', 'C90'], 'UbicacionNodo': {'D1': (np.float64(4.75021190869025), np.float64(-74.08124218159384)), 'D2': (np.float64(4.5363832206427785), np.float64(-74.10993358606953)), 'D3': (np.float64(4.792925960208614), n

In [5]:
# # --- Markdown ---
# # ## 2. Distance and Cost Matrix Calculation using OSRM
# #
# # Define and execute the function provided by the user to compute distances using the OSRM API.
# # Extract the results into the format required by the Pyomo model.
# # --- End Markdown ---

# # --- Python Code ---
# def compute_distance_matrix(data):
#     """
#     Computes the distance matrix using OSRM table service for ground vehicles.
#     Modifies the data['d'] dictionary in place.
#     Input 'data' dictionary requires:
#         - data['N']: List of node IDs
#         - data['UbicacionNodo']: Dict mapping NodeID -> (lat, lon)
#         - data['CV']: List of vehicle types (e.g., ['Gas Car', 'Aereo'])
#         - data['d']: Pre-initialized nested dict {tv: {u: {v: 0}}}
#     """
#     nodes = data['N']
#     node_coords_dict = data['UbicacionNodo']
#     # Extract coords in the order of nodes list for consistent indexing
#     node_coords_list = [node_coords_dict[i] for i in nodes]

#     # Prepare coordinates for the OSRM API in lon,lat format
#     coords_str = ';'.join([f"{lon},{lat}" for lat, lon in node_coords_list])

#     # OSRM distance computation for all ground vehicles (single call)
#     # Exclude 'Aereo' type as per the function's logic
#     ground_vehicle_types = [cv for cv in data['CV'] if cv != 'Aereo']
#     if not ground_vehicle_types:
#          print("Warning: No ground vehicle types found in data['CV'] to calculate distances for.", file=sys.stderr)
#          return # Nothing to calculate

#     url = f"{OSRM_BASE_URL}/table/v1/driving/{coords_str}"
#     print(f"Requesting distances from OSRM: {url[:100]}...")
#     params = {
#         'annotations': 'distance'
#     }
#     try:
#         # Single API call
#         response = requests.get(url, params=params, timeout=90) # Increased timeout
#         response.raise_for_status()  # Raise an HTTPError if status != 200
#         result = response.json()

#         if result['code'] != 'Ok':
#             raise ValueError(f"OSRM API Error: {result['code']} - {result.get('message', 'No message')}")
#         if 'distances' not in result:
#             raise ValueError("No 'distances' key found in OSRM response")

#         distances = result['distances'] # This is a list of lists

#         # Check matrix dimensions
#         if len(distances) != len(nodes) or (len(distances) > 0 and len(distances[0]) != len(nodes)):
#              raise ValueError(f"OSRM returned distance matrix of unexpected size: {len(distances)}x{len(distances[0]) if len(distances)>0 else 0}, expected {len(nodes)}x{len(nodes)}")


#         # Populate the data['d'] dictionary
#         for tv in ground_vehicle_types:
#             for i, from_node in enumerate(nodes):
#                 for j, to_node in enumerate(nodes):
#                     if distances[i] is None or distances[i][j] is None:
#                          print(f"Warning: OSRM returned null distance for ({from_node}, {to_node}). Setting to infinity.", file=sys.stderr)
#                          data['d'][tv][from_node][to_node] = float('inf')
#                     else:
#                         distance = distances[i][j] / 1000.0  # Convert meters to kilometers
#                         data['d'][tv][from_node][to_node] = distance
#         print("OSRM distance calculation successful.")

#     except requests.exceptions.Timeout:
#         print(f"Error: OSRM request timed out.", file=sys.stderr)
#         print("Setting distances to infinity.", file=sys.stderr)
#         for tv in ground_vehicle_types:
#             for i in nodes:
#                 for j in nodes:
#                     data['d'][tv][i][j] = float('inf')
#     except Exception as e:
#         print(f"Error fetching or processing distances for ground vehicles: {e}", file=sys.stderr)
#         print("Setting distances to infinity.", file=sys.stderr)
#         # Default to a large number (infinity) in case of any failure
#         for tv in ground_vehicle_types:
#             for i in nodes:
#                 for j in nodes:
#                     data['d'][tv][i][j] = float('inf')

# # --- Execute Distance Calculation ---
# print("Calculating distances using the provided OSRM function...")
# compute_distance_matrix(data)

# # --- Extract distances into Pyomo parameter format ---
# # Assuming distance is the same for all ground vehicle types in the model
# # We take the distances calculated for the first ground vehicle type
# ground_vehicle_types = [cv for cv in data['CV'] if cv != 'Aereo']
# if ground_vehicle_types:
#     first_ground_type = ground_vehicle_types[0]
#     if first_ground_type in data['d']:
#         for u in data['N']:
#             for v in data['N']:
#                 # Check if the inner dictionary exists before accessing
#                 if u in data['d'][first_ground_type] and v in data['d'][first_ground_type][u]:
#                     param_d_data[u, v] = data['d'][first_ground_type][u][v]
#                     param_c_data[u, v] = C_KM * param_d_data[u, v]
#                 else:
#                     # Handle cases where a node pair might be missing (shouldn't happen with table)
#                     print(f"Warning: Missing distance data for ({u}, {v}). Setting to infinity.", file=sys.stderr)
#                     param_d_data[u, v] = float('inf')
#                     param_c_data[u, v] = float('inf')
#         print(f"Extracted distances for Pyomo model using type '{first_ground_type}'.")
#     else:
#          print(f"Error: No distance data found for vehicle type '{first_ground_type}' after OSRM call. Using zeros.", file=sys.stderr)
#          for u in data['N']:
#             for v in data['N']:
#                 param_d_data[u, v] = 0.0
#                 param_c_data[u, v] = 0.0
# else:
#     print("Error: No ground vehicle types defined. Cannot extract distances. Using zeros.", file=sys.stderr)
#     for u in data['N']:
#         for v in data['N']:
#             param_d_data[u, v] = 0.0
#             param_c_data[u, v] = 0.0

# # Display a small part of the extracted distance matrix for verification
# print("\n--- Sample Extracted Distances (km) for Pyomo ---")
# sample_nodes = set_N_data[:4] # Show first 4 nodes
# for u in sample_nodes:
#     for v in sample_nodes:
#         dist_val = param_d_data.get((u, v), float('inf'))
#         # Check for infinity before formatting
#         dist_str = f"{dist_val:.2f}" if dist_val != float('inf') else "inf"
#         print(f"d({u}, {v}) = {dist_str}", end = ' | ')
#     print()
# print("--- End Distance Sample ---")

# # Reminder about public server limitations
# print("\nNOTE: Using the public OSRM demo server. This has rate limits and is not suitable for large-scale or production use.")
# print("Consider setting up a local OSRM instance for better performance and reliability.")
# # --- End Python Code ---

# # # --- Python Code ---
# # import math, json, os, requests

# # OSRM_URL   = "https://router.project-osrm.org/table/v1/driving/"
# # CACHE_FILE = "osrm_km_cache.json"     # se reutiliza entre ejecuciones
# # C_KM       = 20700                    # COP por km

# # # ------------------------------------------------------------------
# # # 1)  Construir lista de nodos y diccionario de coordenadas
# # #     (ya los tienes como set_N_data y data['UbicacionNodo'])
# # # ------------------------------------------------------------------
# # N      = data['N']
# # coords = data['UbicacionNodo']        # {node: (lat, lon)}

# # # ------------------------------------------------------------------
# # # 2)  Cargar caché (claves "u|v")
# # # ------------------------------------------------------------------
# # dist_cache = {}
# # if os.path.exists(CACHE_FILE):
# #     try:
# #         with open(CACHE_FILE) as f:
# #             raw = json.load(f)
# #         dist_cache = {tuple(k.split("|")): v for k, v in raw.items()}
# #         print(f"  Distancias cargadas de caché ({len(dist_cache)} pares).")
# #     except json.JSONDecodeError:
# #         print("  Caché dañada, se ignorará.")
# #         dist_cache = {}

# # # ------------------------------------------------------------------
# # # 3)  Identificar pares faltantes y, si hay, llamar a OSRM
# # # ------------------------------------------------------------------
# # missing = [(u, v) for u in N for v in N if (u, v) not in dist_cache]

# # if missing:
# #     print(f"Consultando OSRM para {len(missing)} pares…")
# #     MAX_COORDS = 100                                   # límite del servidor demo
# #     for start in range(0, len(N), MAX_COORDS):
# #         sub_nodes  = N[start:start+MAX_COORDS]
# #         coord_str  = ';'.join(f"{coords[n][1]},{coords[n][0]}" for n in sub_nodes)
# #         try:
# #             r = requests.get(OSRM_URL + coord_str,
# #                              params={"annotations": "distance"},
# #                              timeout=60)
# #             r.raise_for_status()
# #             matrix = r.json()["distances"]             # metros
# #         except Exception as e:
# #             print("  Error OSRM:", e, "→ se usa ∞")
# #             matrix = [[math.inf]*len(sub_nodes) for _ in sub_nodes]

# #         for i, u in enumerate(sub_nodes):
# #             for j, v in enumerate(sub_nodes):
# #                 dist_cache[(u, v)] = matrix[i][j] / 1000.0   # a km

# #     # guardar caché
# #     with open(CACHE_FILE, "w") as f:
# #         json.dump({f"{u}|{v}": km for (u, v), km in dist_cache.items()}, f)
# #     print(" Caché OSRM actualizada.")
# # else:
# #     print("No faltaban distancias nuevas.")

# # # ------------------------------------------------------------------
# # # 4)  Rellenar param_d_data y param_c_data para Pyomo
# # # ------------------------------------------------------------------
# # param_d_data.clear()
# # param_c_data.clear()
# # for u in N:
# #     for v in N:
# #         km = dist_cache.get((u, v), math.inf)
# #         param_d_data[u, v] = km
# #         param_c_data[u, v] = km * C_KM
# # print(f"➜  Matriz de distancias y costos lista ({len(N)} × {len(N)} pares).")

# # # (opcional) mostrar un subconjunto
# # print("\nMuestra (primeros 4 nodos):")
# # for u in N[:4]:
# #     for v in N[:4]:
# #         print(f"d({u},{v})={param_d_data[u,v]:.2f} km, "
# #               f"c={param_c_data[u,v]:,.0f}", end="  |  ")
# #     print()
# # # --- End Python Code ---.

# --- Markdown ---
# ## 2. Distance and Cost Matrix Calculation using OSRM (Batched & Simplified)
#
# Calculate the distance matrix using OSRM, handling potential coordinate limits by batching requests.
# Incorporates caching to avoid redundant API calls.
# --- End Markdown ---

# --- Markdown ---
# ## 2. Distance and Cost Matrix Calculation using OSRM (Batched & Simplified)
#
# Calculate the distance matrix using OSRM, handling potential coordinate limits by batching requests.
# Incorporates caching to avoid redundant API calls.
# --- End Markdown ---

# --- Python Code ---
import math
import json
import os
import requests
import time
import sys

node_coords_dict = {node_id: (lat, lon) for node_id, (lat, lon) in data['UbicacionNodo'].items()}
set_N_data = data['N'] # List of node IDs

OSRM_BASE_URL = "http://router.project-osrm.org"
CACHE_FILE = "osrm_km_cache_pyomo.json"  # Cache file specific to this script
C_KM = 20700                            # COP per km
MAX_OSRM_COORDS = 99                    # Public server limit is often 100, use 99 for safety

def compute_osrm_distance_matrix_batched(node_ids, node_coords_dict):
    """
    Computes the distance matrix using OSRM table service, handling coordinate limits via batching.
    Uses a cache file to store and retrieve previously calculated distances.

    Args:
        node_ids (list): List of node IDs (e.g., ['D1', 'C1', 'C2', ...]).
        node_coords_dict (dict): Dictionary mapping NodeID -> (latitude, longitude).

    Returns:
        dict: A dictionary representing the distance matrix {(from_node, to_node): distance_km}.
              Returns None if critical errors occur.
    """
    N = len(node_ids)
    dist_matrix_km = {}

    # --- 1. Caching ---
    dist_cache = {}
    if os.path.exists(CACHE_FILE):
        try:
            with open(CACHE_FILE) as f:
                dist_cache = json.load(f)
            print(f"  Distances loaded from cache ({len(dist_cache)} pairs).")
        except json.JSONDecodeError:
            print("  Cache file damaged or empty, will rebuild.", file=sys.stderr)
            dist_cache = {}
        except Exception as e:
            print(f"  Error loading cache: {e}. Will rebuild.", file=sys.stderr)
            dist_cache = {}

    # --- 2. Identify Missing Pairs and Prepare Batches ---
    coords_for_osrm = {} # Dict {node_id: "lon,lat"}
    nodes_in_batches = {} # Dict {node_id: batch_index}
    batches = []          # List of lists of node IDs

    # Create coordinate strings and identify nodes needing calculation
    needs_calculation = False
    for u in node_ids:
        coords_for_osrm[u] = f"{node_coords_dict[u][1]},{node_coords_dict[u][0]}" # lon,lat
        for v in node_ids:
            if u == v:
                dist_matrix_km[(u,v)] = 0.0 # Distance to self is 0
            elif f"{u}|{v}" not in dist_cache:
                needs_calculation = True
                # Assign nodes to batches if not already done
                if u not in nodes_in_batches:
                    if not batches or len(batches[-1]) >= MAX_OSRM_COORDS:
                        batches.append([])
                    batches[-1].append(u)
                    nodes_in_batches[u] = len(batches) - 1
                if v not in nodes_in_batches:
                    if not batches or len(batches[-1]) >= MAX_OSRM_COORDS:
                         batches.append([])
                    batches[-1].append(v)
                    nodes_in_batches[v] = len(batches) - 1
            else:
                # Load from cache
                 dist_matrix_km[(u,v)] = dist_cache[f"{u}|{v}"]


    if not needs_calculation:
        print("All required distances found in cache.")
        return dist_matrix_km

    print(f"Need to calculate distances. Grouping {len(nodes_in_batches)} unique nodes into {len(batches)} batches of up to {MAX_OSRM_COORDS}.")

    # --- 3. OSRM API Calls (Batched) ---
    new_cache_entries = 0
    for i in range(len(batches)):
        # For each batch, get distances from ALL nodes TO nodes in this batch
        # This strategy minimizes API calls compared to batch x batch
        batch_nodes = batches[i]
        batch_coords_str = ";".join(coords_for_osrm[n] for n in batch_nodes)
        all_coords_str = ";".join(coords_for_osrm[n] for n in node_ids) # All nodes as potential sources

        # Define source and destination indices for the API call
        # Sources: all nodes (indices 0 to N-1)
        # Destinations: nodes in the current batch (indices within all_coords_str)
        dest_indices_map = {node_id: idx for idx, node_id in enumerate(node_ids)}
        dest_indices_str = ";".join(str(dest_indices_map[n]) for n in batch_nodes)

        url = f"{OSRM_BASE_URL}/table/v1/driving/{all_coords_str}"
        params = {
            'annotations': 'distance',
            'destinations': dest_indices_str # Calculate distances TO this batch
            # 'sources' defaults to all if not specified
        }
        print(f"  Requesting batch {i+1}/{len(batches)}: distances from all {N} nodes TO {len(batch_nodes)} nodes...")

        try:
            response = requests.get(url, params=params, timeout=120) # Longer timeout for potentially larger requests
            response.raise_for_status()
            result = response.json()

            if result['code'] != 'Ok':
                print(f"  ERROR: OSRM API Error for batch {i+1}: {result['code']} - {result.get('message', 'No message')}", file=sys.stderr)
                continue # Try next batch, affected pairs will remain infinity

            osrm_distances = result['distances'] # Matrix: sources x destinations_in_batch

            # Check dimensions carefully
            if len(osrm_distances) != N or (len(osrm_distances)>0 and len(osrm_distances[0]) != len(batch_nodes)):
                print(f"  ERROR: OSRM returned matrix of unexpected size for batch {i+1}. Skipping batch.", file=sys.stderr)
                continue

            # Populate cache and the main distance matrix
            for src_idx, u in enumerate(node_ids):
                for dest_idx, v in enumerate(batch_nodes):
                    # Only update if missing or cache needs refresh (optional)
                    cache_key = f"{u}|{v}"
                    if u == v: continue # Already handled distance to self

                    if cache_key not in dist_cache:
                        if osrm_distances[src_idx] is None or osrm_distances[src_idx][dest_idx] is None:
                            print(f"  Warning: OSRM null distance for ({u}, {v}) in batch {i+1}. Setting cache to infinity.", file=sys.stderr)
                            dist_km = float('inf')
                        else:
                            dist_km = osrm_distances[src_idx][dest_idx] / 1000.0
                        dist_cache[cache_key] = dist_km
                        dist_matrix_km[(u,v)] = dist_km
                        new_cache_entries += 1

            time.sleep(3) # Add delay between batches to respect public server limits

        except requests.exceptions.RequestException as e:
            print(f"  ERROR: OSRM request failed for batch {i+1}: {e}", file=sys.stderr)
        except Exception as e:
            print(f"  Error processing OSRM response for batch {i+1}: {e}", file=sys.stderr)


    # --- 4. Fill remaining missing pairs (if any due to errors) with infinity ---
    for u in node_ids:
        for v in node_ids:
            if (u, v) not in dist_matrix_km:
                 print(f"Warning: Distance ({u}, {v}) could not be calculated. Setting to infinity.", file=sys.stderr)
                 dist_matrix_km[u, v] = float('inf')


    # --- 5. Save updated cache ---
    if new_cache_entries > 0:
        try:
            with open(CACHE_FILE, "w") as f:
                json.dump(dist_cache, f)
            print(f"  Updated OSRM cache ({new_cache_entries} new entries).")
        except Exception as e:
            print(f"  Error saving cache file: {e}", file=sys.stderr)

    print("OSRM distance calculation finished.")
    return dist_matrix_km

# --- Execute Distance Calculation ---
print("Calculating distances using the batched OSRM function...")
# Call the function with the node list and coordinate dictionary



calculated_distances = compute_osrm_distance_matrix_batched(set_N_data, node_coords_dict)

# --- Populate Pyomo parameter dictionaries ---
if calculated_distances is not None:
    param_d_data = calculated_distances
    param_c_data = {} # Reset cost dictionary
    for (u, v), dist_km in param_d_data.items():
        # Assign large cost for infinite distance to penalize in objective
        param_c_data[u, v] = C_KM * dist_km if dist_km != float('inf') else 1e12 # Large cost for infinity
    print("Populated distance and cost parameters for Pyomo.")
else:
    print("Error: Distance calculation failed critically. Using infinite distances/costs.", file=sys.stderr)
    # Fallback to infinity if OSRM call failed completely
    param_d_data = {}
    param_c_data = {}
    for u in set_N_data:
        for v in set_N_data:
            param_d_data[u, v] = 0.0 if u == v else float('inf')
            param_c_data[u, v] = 0.0 if u == v else float('inf')

# Display a small part of the extracted distance matrix for verification
print("\n--- Sample Extracted Distances (km) for Pyomo ---")
sample_nodes = set_N_data[:4] # Show first 4 nodes
for u in sample_nodes:
    for v in sample_nodes:
        dist_val = param_d_data.get((u, v), float('inf'))
        dist_str = f"{dist_val:.2f}" if dist_val != float('inf') else "inf"
        print(f"d({u}, {v}) = {dist_str}", end = ' | ')
    print()
print("--- End Distance Sample ---")

# Reminder about public server limitations
print("\nNOTE: Using the public OSRM demo server. This has rate limits and is not suitable for large-scale or production use.")
print("Consider setting up a local OSRM instance for better performance and reliability.")
# --- End Python Code ---

NameError: name 'data' is not defined

In [3]:
# --- Markdown ---
# ## 3. Pyomo Model Definition
#
# Define the Pyomo model structure (sets, parameters, variables, objective, constraints)
# using the data loaded from CSV and distances calculated via OSRM.
# --- End Markdown ---

# --- Python Code ---
# Create a concrete model
model = pyo.ConcreteModel(name="LogistiCo_VRP_CSV_OSRM_UserFunc")

# --- Sets ---
model.I = pyo.Set(initialize=set_I_data, doc="Distribution Centers")
model.J = pyo.Set(initialize=set_J_data, doc="Customers")
model.K = pyo.Set(initialize=set_K_data, doc="Vehicles")
model.N = pyo.Set(initialize=set_N_data, doc="All Nodes")

# --- Parameters ---
model.A = pyo.Param(model.I, initialize=param_A_data, default=float('inf'), doc="Capacity of CD i")
model.D = pyo.Param(model.J, initialize=param_D_data, doc="Demand of customer j")
model.Q = pyo.Param(model.K, initialize=param_Q_data, doc="Capacity of vehicle k")
model.R = pyo.Param(model.K, initialize=param_R_data, doc="Range of vehicle k")
# Initialize with the extracted OSRM distances, default to infinity if missing
model.d = pyo.Param(model.N, model.N, initialize=param_d_data, default=float('inf'), doc="Distance from u to v (km)")
model.c = pyo.Param(model.N, model.N, initialize=param_c_data, default=float('inf'), doc="Cost from u to v (COP)")
model.n_cust = pyo.Param(initialize=param_n_cust_data, doc="Number of customers")

# --- Decision Variables ---
model.x = pyo.Var(model.N, model.N, model.K, within=pyo.Binary, doc="Vehicle k travels from u to v")
model.y = pyo.Var(model.I, model.K, within=pyo.Binary, doc="Vehicle k assigned to CD i")
model.u = pyo.Var(model.J, model.K, within=pyo.NonNegativeReals, bounds=(0, model.n_cust), doc="MTZ auxiliary variable") # Added bounds for clarity

# --- Objective Function ---
def objective_rule(mod):
    # Ensure cost is not infinite when calculating objective
    cost = sum(mod.c[u, v] * mod.x[u, v, k]
               for k in mod.K for u in mod.N for v in mod.N
               if u != v and mod.c[u, v] != float('inf'))
    # Add penalty for using arcs with infinite cost (should not happen if feasible)
    penalty = sum(1e9 * mod.x[u, v, k]
                   for k in mod.K for u in mod.N for v in mod.N
                   if u != v and mod.c[u, v] == float('inf'))
    return cost + penalty
model.objective = pyo.Objective(rule=objective_rule, sense=pyo.minimize, doc="Minimize total cost")

# --- Constraints ---
# Constraint 1: Customer Coverage
def customer_coverage_rule(mod, j):
    return sum(mod.x[u, j, k] for k in mod.K for u in mod.N if u != j) == 1
model.customer_coverage = pyo.Constraint(model.J, rule=customer_coverage_rule, doc="Each customer visited exactly once")

# Constraint 2a: Flow Conservation at Customer Nodes
def flow_conservation_customer_rule(mod, j, k):
    in_flow = sum(mod.x[u, j, k] for u in mod.N if u != j)
    out_flow = sum(mod.x[j, v, k] for v in mod.N if v != j)
    return in_flow == out_flow
model.flow_conservation_customer = pyo.Constraint(model.J, model.K, rule=flow_conservation_customer_rule, doc="Flow conservation at customer nodes")

# Constraint 2b: Vehicle Departure from Assigned Depot
def vehicle_departure_rule(mod, i, k):
    return sum(mod.x[i, v, k] for v in mod.N if v != i) == mod.y[i, k]
model.vehicle_departure = pyo.Constraint(model.I, model.K, rule=vehicle_departure_rule, doc="Vehicle leaves assigned depot")

# Constraint 2c: Vehicle Return to Assigned Depot
def vehicle_return_rule(mod, i, k):
    return sum(mod.x[u, i, k] for u in mod.N if u != i) == mod.y[i, k]
model.vehicle_return = pyo.Constraint(model.I, model.K, rule=vehicle_return_rule, doc="Vehicle returns to assigned depot")

# Constraint 2d: Vehicle Assignment Constraint
def vehicle_assignment_rule(mod, k):
    return sum(mod.y[i, k] for i in mod.I) <= 1
model.vehicle_assignment = pyo.Constraint(model.K, rule=vehicle_assignment_rule, doc="Each vehicle assigned to at most one depot")


# Constraint 3: Vehicle Capacity
def vehicle_capacity_rule(mod, k):
    demand_served_by_k = sum(mod.D[j] * sum(mod.x[u, j, k] for u in mod.N if u != j) for j in mod.J)
    return demand_served_by_k <= mod.Q[k]
model.vehicle_capacity = pyo.Constraint(model.K, rule=vehicle_capacity_rule, doc="Vehicle capacity constraint")

# Constraint 4: Vehicle Range (Autonomy)
def vehicle_range_rule(mod, k):
    # Ensure distance is not infinite when calculating range
    dist_traveled = sum(mod.d[u, v] * mod.x[u, v, k]
                       for u in mod.N for v in mod.N
                       if u != v and mod.d[u,v] != float('inf'))
    # Add a check to ensure no infeasible arcs are used
    infeasible_arc_used = sum(mod.x[u, v, k]
                              for u in mod.N for v in mod.N
                              if u != v and mod.d[u,v] == float('inf'))
    if pyo.value(infeasible_arc_used) > 0.1: # If solver uses an infinite arc
         return pyo.Constraint.Infeasible
    return dist_traveled <= mod.R[k]
model.vehicle_range = pyo.Constraint(model.K, rule=vehicle_range_rule, doc="Vehicle range constraint")

# Constraint 5: Distribution Center Capacity (Omitted)
print("Skipping CD Capacity Constraint (5).")

# Constraint 6: Subtour Elimination (MTZ)
def subtour_elimination_rule(mod, j, j_prime, k):
    if j == j_prime:
        return pyo.Constraint.Skip
    # Make sure arc exists before applying constraint
    if mod.d[j,j_prime] == float('inf'): # If no route exists between j and j', x should be 0
        return mod.x[j,j_prime,k] == 0
    return mod.u[j, k] - mod.u[j_prime, k] + mod.n_cust * mod.x[j, j_prime, k] <= mod.n_cust - 1
model.subtour_elimination = pyo.Constraint(model.J, model.J, model.K, rule=subtour_elimination_rule, doc="MTZ subtour elimination")

# Constraint 7: Prevent trivial loops (x_uuk = 0)
def no_trivial_loops_rule(mod, u, k):
    return mod.x[u, u, k] == 0
model.no_trivial_loops = pyo.Constraint(model.N, model.K, rule=no_trivial_loops_rule, doc="Prevent travel from a node to itself")

print("Pyomo model structure defined using data from CSVs and OSRM API (User Func).")
# --- End Python Code ---

Skipping CD Capacity Constraint (5).
Pyomo model structure defined using data from CSVs and OSRM API (User Func).


In [ ]:
# --- Markdown ---
# ## 4. Solve the Model and Display Results
#
# Use a selected MILP solver (e.g., GLPK, CBC) to find the optimal solution
# for the VRP model and display the results, including cost, assignments, and routes.
# --- End Markdown ---

# --- Python Code ---
solver_name = 'gurobi' # Or 'cbc'
try:
    solver = pyo.SolverFactory(solver_name)
    if not solver.available():
        raise RuntimeError(f"Solver '{solver_name}' not found or not executable.")
except Exception as e:
    print(f"ERROR: Could not find or initialize solver '{solver_name}'. Please install it and ensure it's in your PATH. Error: {e}", file=sys.stderr)
    print("You can try installing GLPK or CBC (e.g., using conda).", file=sys.stderr)
    sys.exit(1)


print(f"\nSolving the model using {solver_name}...")
# Add a time limit, e.g., 5 minutes, especially for larger problems
# results = solver.solve(model, tee=True, timelimit=300)
results = solver.solve(model, tee=True) # tee=True shows solver output

# --- Display Results ---

print("\n--- Solver Results ---")
print(results) # Print detailed solver results object

if results.solver.termination_condition == pyo.TerminationCondition.optimal or \
   (results.solver.termination_condition == pyo.TerminationCondition.feasible and len(results.solution) > 0) or \
   (results.solver.termination_condition == pyo.TerminationCondition.maxTimeLimit and len(results.solution) > 0):

    if results.solver.termination_condition == pyo.TerminationCondition.optimal:
        print("\n--- Optimal Solution Found ---")
    elif results.solver.termination_condition == pyo.TerminationCondition.maxTimeLimit:
         print("\n--- Time Limit Reached - Feasible Solution Found ---")
    else:
        print("\n--- Feasible Solution Found (may not be optimal) ---")

    print(f"Minimum Total Cost: {pyo.value(model.objective):,.2f} COP")

    print("\n--- Vehicle Assignments (y_ik) ---")
    assigned_vehicles_count = 0
    assignments = {}
    for k in model.K:
        assignments[k] = 'Unassigned'
        for i in model.I:
            # Use a tolerance for checking binary variable values
            if pyo.value(model.y[i, k], exception=False) is not None and pyo.value(model.y[i, k]) > 0.5:
                print(f"Vehicle {k} starts from CD {i}")
                assignments[k] = i
                assigned_vehicles_count += 1
                break
        if assignments[k] == 'Unassigned':
             print(f"Vehicle {k} is not used.")
    if assigned_vehicles_count == 0:
        print("No vehicles were assigned to any routes.")

    print("\n--- Routes (x_uvk) ---")
    total_distance = 0
    total_demand_served = 0
    vehicles_used = set()

    for k in model.K:
        start_depot = assignments.get(k)
        if start_depot != 'Unassigned':
            print(f"\nRoute for Vehicle {k} (from {start_depot}):")
            route = [start_depot]
            current_node = start_depot
            route_distance = 0
            route_demand = 0
            visited_nodes = {start_depot}
            route_found = False
            max_steps = len(model.N) + 2 # Increased safety margin

            for step in range(max_steps):
                next_node_found = False
                possible_next = []
                for v in model.N:
                    if current_node != v:
                         # Check value safely, handling potential None if variable wasn't solved
                         x_val = pyo.value(model.x[current_node, v, k], exception=False)
                         if x_val is not None and x_val > 0.5:
                            possible_next.append(v)
                            next_node_found = True

                if len(possible_next) > 1 and current_node != start_depot:
                     print(f"  ERROR: Multiple next steps found from {current_node} for vehicle {k}: {possible_next}")
                     route = [start_depot, "Error"] # Mark route as problematic
                     break
                elif not next_node_found and current_node != start_depot:
                     print(f"  ERROR: No next step found from {current_node} for vehicle {k}. Incomplete route.")
                     route.append("Error")
                     break
                elif not next_node_found and current_node == start_depot:
                    print(f"  Vehicle {k} assigned but seems to have no outgoing route from {start_depot}.")
                    route = [start_depot]
                    break

                if next_node_found:
                    next_node = possible_next[0]
                    arc_dist = model.d[current_node, next_node]
                    if arc_dist == float('inf'):
                        print(f"  ERROR: Route uses an infeasible arc from {current_node} to {next_node}")
                        route.append(f"{next_node}(inf)")
                        route_found = False # Mark as incomplete due to error
                        break
                    route_distance += arc_dist
                    route.append(next_node)
                    visited_nodes.add(next_node)

                    if next_node in model.J:
                        route_demand += model.D[next_node]

                    current_node = next_node

                    if current_node == start_depot:
                        route_found = True
                        break

                if step == max_steps - 1 and not route_found:
                    print(f"  ERROR: Route for vehicle {k} did not return to depot within {max_steps} steps.")
                    route.append("...") # Indicate incomplete


            # Print reconstructed route and details
            if route_found:
                 print(f"  {' -> '.join(route)}")
                 print(f"  Distance: {route_distance:.2f} km (Max: {model.R[k]:.2f})")
                 print(f"  Demand: {route_demand:.2f} kg (Capacity: {model.Q[k]:.2f})")
                 total_distance += route_distance
                 total_demand_served += route_demand
                 vehicles_used.add(k)
                 # Validation checks
                 if route_distance > model.R[k] + 1e-6: print(f"  WARNING: Vehicle {k} exceeded range!")
                 if route_demand > model.Q[k] + 1e-6: print(f"  WARNING: Vehicle {k} exceeded capacity!")
            elif len(route) > 1 : # Print incomplete/error routes if they exist
                 print(f"  Route Issue: {' -> '.join(route)}")
            # Else: already handled cases where vehicle didn't leave depot


    print("\n--- Overall Summary ---")
    print(f"Total distance covered by all vehicles: {total_distance:.2f} km")
    print(f"Total demand served: {total_demand_served:.2f} kg")
    print(f"Total expected demand: {sum(param_D_data.values()):.2f} kg")
    print(f"Number of vehicles used: {len(vehicles_used)} out of {len(model.K)}")
    if abs(total_demand_served - sum(param_D_data.values())) > 1e-6:
        print("WARNING: Total demand served does not match total expected demand!")


elif results.solver.termination_condition == pyo.TerminationCondition.infeasible:
     print("\n--- Model is Infeasible ---")
     print("The solver determined that there is no solution that satisfies all constraints.")
     print("Check constraints (capacities, ranges) and data for potential issues.")
     # Consider using Pyomo's tools for analyzing infeasibility if needed:
     # from pyomo.util.infeasible import log_infeasible_constraints
     # log_infeasible_constraints(model)

else:
    print("\n--- Solver did not find an Optimal or Feasible Solution ---")
    print(f"Solver Status: {results.solver.status}")
    print(f"Termination Condition: {results.solver.termination_condition}")
    print("Check solver logs and model formulation for errors.")
# --- End Python Code ---


Solving the model using gurobi...
Set parameter Username
Set parameter LicenseID to value 2654934
Academic license - for non-commercial use only - expires 2026-04-21
Read LP format model from file C:\Users\VivoBook\AppData\Local\Temp\tmp5n_6xmwc.pyomo.lp
Reading time = 2.93 seconds
x1: 246960 rows, 315180 columns, 2197620 nonzeros
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i7-8565U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 246960 rows, 315180 columns and 2197620 nonzeros
Model fingerprint: 0x23ca9ff5
Variable types: 2700 continuous, 312480 integer (312480 binary)
Coefficient statistics:
  Matrix range     [2e-01, 9e+01]
  Objective range  [4e+03, 9e+05]
  Bounds range     [1e+00, 9e+01]
  RHS range        [1e+00, 1e+03]
Presolve removed 3060 rows and 82565 columns (presolve time = 5s)...
Presolve removed 3418 rows 

# Version 2 para carga de datos y poder calcular la matriz de distancia con batchs


In [ ]:
# --- Python Code ---
import pyomo.environ as pyo
import pandas as pd
import requests   # Para llamadas HTTP al API OSRM
import json       # Para parsear respuestas JSON
import time       # Para añadir delays (si es necesario)
import sys        # Para mensajes de error
import math       # Para math.inf
import os         # Para manejar archivos (caché)

# --- Configuración ---
OSRM_BASE_URL = "http://router.project-osrm.org"
CACHE_FILE = "osrm_km_cache_pyomo_batched.json" # Archivo caché específico
C_KM = 20700                                    # COP por km
# Límite práctico para el servidor público demo de OSRM (recomendado < 100)


# --- Cargar Datos de CSV ---
try:
    # Ajusta las rutas si tus archivos están en otra carpeta
    depots_df = pd.read_csv("case_3_base/Depots.csv")
    clients_df = pd.read_csv("case_3_base/Clients.csv")
    vehicles_df = pd.read_csv("case_3_base/Vehicles.csv")
    print("Archivos CSV cargados exitosamente.")
except FileNotFoundError as e:
    print(f"ERROR: No se encontró el archivo CSV: {e}. Asegúrate que están en la carpeta correcta.", file=sys.stderr)
    sys.exit(1)
except Exception as e:
    print(f"ERROR: Ocurrió un error al cargar los archivos CSV: {e}", file=sys.stderr)
    sys.exit(1)

# --- Procesar Datos para Pyomo ---

# A. Sets de Pyomo
set_I_data = [f'D{depot_id}' for depot_id in depots_df['DepotID']]
set_J_data = [f'C{client_id}' for client_id in clients_df['ClientID']]
set_K_data = [f'V{i+1}' for i in range(len(vehicles_df))]
set_N_data = set_I_data + set_J_data # Lista ordenada de nodos

# B. Parámetros de Pyomo (excluyendo distancias/costos por ahora)
param_A_data = {f'D{depots_df.loc[i, "DepotID"]}': float('inf') for i in depots_df.index}
print("AVISO: Capacidades de depósito (A_i) no encontradas en Depots.csv. Asumiendo capacidad infinita.")

param_D_data = {f'C{clients_df.loc[i, "ClientID"]}': clients_df.loc[i, 'Product']
                for i in clients_df.index}
param_Q_data = {f'V{i+1}': vehicles_df.loc[i, 'Capacity'] for i in vehicles_df.index}
param_R_data = {f'V{i+1}': vehicles_df.loc[i, 'Range'] for i in vehicles_df.index}
param_n_cust_data = len(set_J_data)

# C. Diccionario de Coordenadas (NodeID -> (lat, lon)) - Necesario para OSRM
node_coords_dict = {}
for i in depots_df.index:
    node_id = f'D{depots_df.loc[i, "DepotID"]}'
    node_coords_dict[node_id] = (depots_df.loc[i, 'Latitude'], depots_df.loc[i, 'Longitude'])
for i in clients_df.index:
    node_id = f'C{clients_df.loc[i, "ClientID"]}'
    node_coords_dict[node_id] = (clients_df.loc[i, 'Latitude'], clients_df.loc[i, 'Longitude'])

# D. Parámetros Placeholder para Pyomo (se llenarán con OSRM)
param_d_data = {} # Almacenará {(u, v): distance_km}
param_c_data = {} # Almacenará {(u, v): cost_cop}

print("--- Datos Preparados para Pyomo y Función de Distancia ---")
print(f"Set I (Depots): {set_I_data}")
print(f"Set J (Customers): {set_J_data}")
print(f"Set K (Vehicles): {set_K_data}")
print(f"Set N (Nodes): {set_N_data}")
print(f"Número de nodos: {len(set_N_data)}") # Info útil
print(f"Param D (Demands): {param_D_data}")
print(f"Param Q (Veh. Caps): {param_Q_data}")
print(f"Param R (Veh. Ranges): {param_R_data}")
print(f"Param n_cust: {param_n_cust_data}")
# print(f"Node Locations (for OSRM): {node_coords_dict}") # Puede ser largo
print("--- Fin Preparación de Datos ---")
# --- End Python Code ---

Archivos CSV cargados exitosamente.
AVISO: Capacidades de depósito (A_i) no encontradas en Depots.csv. Asumiendo capacidad infinita.
--- Datos Preparados para Pyomo y Función de Distancia ---
Set I (Depots): ['D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12']
Set J (Customers): ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C23', 'C24', 'C25', 'C26', 'C27', 'C28', 'C29', 'C30', 'C31', 'C32', 'C33', 'C34', 'C35', 'C36', 'C37', 'C38', 'C39', 'C40', 'C41', 'C42', 'C43', 'C44', 'C45', 'C46', 'C47', 'C48', 'C49', 'C50', 'C51', 'C52', 'C53', 'C54', 'C55', 'C56', 'C57', 'C58', 'C59', 'C60', 'C61', 'C62', 'C63', 'C64', 'C65', 'C66', 'C67', 'C68', 'C69', 'C70', 'C71', 'C72', 'C73', 'C74', 'C75', 'C76', 'C77', 'C78', 'C79', 'C80', 'C81', 'C82', 'C83', 'C84', 'C85', 'C86', 'C87', 'C88', 'C89', 'C90']
Set K (Vehicles): ['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V

In [2]:
# --- Markdown ---
# ## 2. Cálculo de Matriz de Distancias y Costos usando OSRM (Batching Correcto y Reducido)
#
# Se define y ejecuta la función para calcular distancias usando el servicio `table` de OSRM API,
# manejando el límite de coordenadas mediante batches más pequeños y usando caché.
# Los resultados se extraen al formato requerido por Pyomo.
# --- End Markdown ---

# --- Python Code ---
import math
import json
import os
import requests
import time
import sys

OSRM_BASE_URL = "http://router.project-osrm.org"
CACHE_FILE = "osrm_km_cache_pyomo2.json" # Archivo caché específico
C_KM = 20700                                    # COP por km
# --- REDUCIR TAMAÑO DE BATCH ---
MAX_OSRM_COORDS_PER_BATCH = 15                  # Límite reducido para el servidor demo
# -------------------------------

def compute_osrm_distance_matrix_batched(node_ids, node_coords_dict):
    """
    Calcula la matriz de distancias usando OSRM table service, manejando límites
    de coordenadas via batching y usando caché.

    Args:
        node_ids (list): Lista ordenada de IDs de nodos (e.g., ['D1', 'C1', 'C2', ...]).
        node_coords_dict (dict): Diccionario {NodeID -> (lat, lon)}.

    Returns:
        dict: Diccionario de la matriz de distancias {(nodo_origen, nodo_destino): dist_km}.
              Retorna None si ocurren errores críticos.
    """
    N = len(node_ids)
    dist_matrix_km = {} # Diccionario para almacenar los resultados finales

    # --- 1. Caching (Cargar caché existente) ---
    dist_cache = {}
    if os.path.exists(CACHE_FILE):
        try:
            with open(CACHE_FILE, 'r') as f:
                dist_cache = json.load(f)
            print(f"  Distancias cargadas desde caché ({len(dist_cache)} pares).")
        except (json.JSONDecodeError, IOError) as e:
            print(f"  Advertencia: No se pudo leer el caché ({e}). Se reconstruirá.", file=sys.stderr)
            dist_cache = {}

    # --- 2. Inicializar matriz y identificar pares faltantes ---
    missing_pairs_indices = [] # Almacenará tuplas de índices (i, j)
    node_index_map = {node_id: i for i, node_id in enumerate(node_ids)} # Mapeo ID -> índice

    for i, u in enumerate(node_ids):
        for j, v in enumerate(node_ids):
            if u == v:
                dist_matrix_km[(u, v)] = 0.0
            else:
                cache_key = f"{u}|{v}"
                if cache_key in dist_cache:
                    # Asegurarse de que el valor del caché es numérico o inf
                    cached_val = dist_cache[cache_key]
                    dist_matrix_km[(u, v)] = float(cached_val) if isinstance(cached_val, (int, float)) else float('inf')
                else:
                    missing_pairs_indices.append((i, j))
                    dist_matrix_km[(u, v)] = float('inf') # Marcar como faltante inicialmente

    if not missing_pairs_indices:
        print("Todas las distancias requeridas encontradas en caché.")
        return dist_matrix_km

    print(f"Se necesitan calcular distancias para {len(missing_pairs_indices)} pares faltantes.")

    # --- 3. Preparar Coordenadas y Batches ---
    coords_for_osrm_list = [(node_coords_dict[n][1], node_coords_dict[n][0]) for n in node_ids] # lon,lat
    coords_for_osrm_str_list = [f"{lon},{lat}" for lon, lat in coords_for_osrm_list]

    indices = list(range(N))
    index_batches = [indices[i:i + MAX_OSRM_COORDS_PER_BATCH]
                     for i in range(0, N, MAX_OSRM_COORDS_PER_BATCH)]
    num_batches = len(index_batches)
    print(f"Nodos agrupados en {num_batches} batches de hasta {MAX_OSRM_COORDS_PER_BATCH} nodos.")

    # --- 4. Llamadas API OSRM (Batch Origen x Batch Destino) ---
    new_cache_entries = 0
    total_calls = num_batches * num_batches
    calls_made_this_run = 0

    for i_batch_idx, src_indices in enumerate(index_batches):
        for j_batch_idx, dest_indices in enumerate(index_batches):

            needs_call = any(f"{node_ids[src_idx]}|{node_ids[dest_idx]}" not in dist_cache
                             for src_idx in src_indices for dest_idx in dest_indices if src_idx != dest_idx)

            if not needs_call:
                continue

            calls_made_this_run += 1
            print(f"  Llamada API {calls_made_this_run}: Origenes batch {i_batch_idx+1}, Destinos batch {j_batch_idx+1}...")

            sources_param = ";".join(map(str, src_indices))
            destinations_param = ";".join(map(str, dest_indices))
            all_coords_str = ";".join(coords_for_osrm_str_list) # URL siempre necesita todas las coords base

            url = f"{OSRM_BASE_URL}/table/v1/driving/{all_coords_str}"
            params = {
                'annotations': 'distance',
                'sources': sources_param,
                'destinations': destinations_param
            }

            try:
                response = requests.get(url, params=params, timeout=180) # Timeout aún más largo por si acaso
                response.raise_for_status()
                result = response.json()

                if result['code'] != 'Ok':
                    print(f"  ERROR: OSRM API Error en batch pair ({i_batch_idx+1}, {j_batch_idx+1}): {result['code']} - {result.get('message', 'No message')}", file=sys.stderr)
                    # Marcar pares como inf si falla la llamada para este batch
                    for u_idx in src_indices:
                        for v_idx in dest_indices:
                            if (u_idx, v_idx) in missing_pairs_indices:
                                u_node, v_node = node_ids[u_idx], node_ids[v_idx]
                                dist_cache[f"{u_node}|{v_node}"] = float('inf')
                                dist_matrix_km[(u_node, v_node)] = float('inf')
                    continue # Saltar al siguiente batch

                osrm_distances = result.get('distances')

                if not osrm_distances or len(osrm_distances) != len(src_indices) or \
                   (len(osrm_distances) > 0 and len(osrm_distances[0]) != len(dest_indices)):
                    print(f"  ERROR: OSRM devolvió matriz de tamaño inesperado para batch pair ({i_batch_idx+1}, {j_batch_idx+1}).", file=sys.stderr)
                    # Marcar pares como inf
                    for u_idx in src_indices:
                        for v_idx in dest_indices:
                             if (u_idx, v_idx) in missing_pairs_indices:
                                 u_node, v_node = node_ids[u_idx], node_ids[v_idx]
                                 dist_cache[f"{u_node}|{v_node}"] = float('inf')
                                 dist_matrix_km[(u_node, v_node)] = float('inf')
                    continue

                # Poblar caché y matriz
                for row_idx_in_batch, u_global_idx in enumerate(src_indices):
                    for col_idx_in_batch, v_global_idx in enumerate(dest_indices):
                        u, v = node_ids[u_global_idx], node_ids[v_global_idx]
                        cache_key = f"{u}|{v}"

                        if u == v: continue

                        # Solo añadir/actualizar si realmente faltaba
                        if (u_global_idx, v_global_idx) in missing_pairs_indices:
                            dist_meters = osrm_distances[row_idx_in_batch][col_idx_in_batch]
                            if dist_meters is None:
                                dist_km = float('inf')
                            else:
                                dist_km = dist_meters / 1000.0

                            dist_cache[cache_key] = dist_km
                            dist_matrix_km[(u, v)] = dist_km
                            new_cache_entries += 1

                # --- AUMENTAR DELAY ---
                time.sleep(3.0) # Delay más largo
                # -----------------------

            except requests.exceptions.RequestException as e:
                print(f"  ERROR: Falló petición OSRM para batch pair ({i_batch_idx+1}, {j_batch_idx+1}): {e}", file=sys.stderr)
                # Marcar pares como inf
                for u_idx in src_indices:
                     for v_idx in dest_indices:
                         if (u_idx, v_idx) in missing_pairs_indices:
                            u_node, v_node = node_ids[u_idx], node_ids[v_idx]
                            dist_cache[f"{u_node}|{v_node}"] = float('inf')
                            dist_matrix_km[(u_node, v_node)] = float('inf')
            except Exception as e:
                 print(f"  Error procesando respuesta OSRM para batch pair ({i_batch_idx+1}, {j_batch_idx+1}): {e}", file=sys.stderr)
                 # Marcar pares como inf
                 for u_idx in src_indices:
                     for v_idx in dest_indices:
                         if (u_idx, v_idx) in missing_pairs_indices:
                            u_node, v_node = node_ids[u_idx], node_ids[v_idx]
                            dist_cache[f"{u_node}|{v_node}"] = float('inf')
                            dist_matrix_km[(u_node, v_node)] = float('inf')


    # --- 5. Rellenar pares faltantes finales ---
    final_missing_count = 0
    for i, u in enumerate(node_ids):
        for j, v in enumerate(node_ids):
             if (u,v) not in dist_matrix_km: # Si aún falta después de los bucles
                  if u!=v:
                     dist_matrix_km[(u, v)] = float('inf')
                     dist_cache[f"{u}|{v}"] = float('inf')
                     final_missing_count +=1
                  else:
                      dist_matrix_km[(u, v)] = 0.0 # Asegurar que la diagonal es 0
    if final_missing_count > 0:
        print(f"Advertencia: {final_missing_count} pares no pudieron ser calculados o cacheados y se marcaron como infinitos.")

    # --- 6. Guardar caché actualizado ---
    if new_cache_entries > 0:
        try:
            with open(CACHE_FILE, "w") as f:
                json.dump(dist_cache, f, indent=2)
            print(f"  Caché OSRM actualizado ({new_cache_entries} nuevas entradas).")
        except Exception as e:
            print(f"  Error guardando archivo caché: {e}", file=sys.stderr)

    print("Cálculo de distancias OSRM terminado.")
    return dist_matrix_km

# --- Ejecutar Cálculo de Distancia ---
print("Calculando distancias usando la función OSRM con batching correcto...")
calculated_distances = compute_osrm_distance_matrix_batched(set_N_data, node_coords_dict)

# --- Poblar parámetros de Pyomo ---
if calculated_distances:
    param_d_data = calculated_distances
    param_c_data = {} # Reiniciar diccionario de costos
    infinity_count = 0
    for (u, v), dist_km in param_d_data.items():
        if dist_km == float('inf'):
            param_c_data[u, v] = float('inf')
            infinity_count += 1
        else:
            param_c_data[u, v] = C_KM * dist_km
    print("Parámetros de distancia y costo poblados para Pyomo.")
    if infinity_count > 0:
        print(f"Advertencia: {infinity_count} pares de nodos tienen distancia infinita (no conectables por OSRM o error).")
else:
    print("Error: El cálculo de distancia falló críticamente. Usando costos/distancias infinitas.", file=sys.stderr)
    param_d_data = {}
    param_c_data = {}
    for u in set_N_data:
        for v in set_N_data:
            param_d_data[u, v] = 0.0 if u == v else float('inf')
            param_c_data[u, v] = 0.0 if u == v else float('inf')

# (Opcional) Mostrar muestra de la matriz
print("\n--- Muestra de Distancias Extraídas (km) para Pyomo ---")
sample_nodes = set_N_data[:4]
for u in sample_nodes:
    for v in sample_nodes:
        dist_val = param_d_data.get((u, v), float('inf'))
        dist_str = f"{dist_val:.2f}" if dist_val != float('inf') else "inf"
        print(f"d({u}, {v}) = {dist_str}", end = ' | ')
    print()
print("--- Fin Muestra de Distancias ---")

print("\nNOTA: Usando el servidor público demo de OSRM. Tiene límites de uso y no es adecuado para producción a gran escala.")
# --- End Python Code ---

Calculando distancias usando la función OSRM con batching correcto...
Se necesitan calcular distancias para 10302 pares faltantes.
Nodos agrupados en 7 batches de hasta 15 nodos.
  Llamada API 1: Origenes batch 1, Destinos batch 1...
  Llamada API 2: Origenes batch 1, Destinos batch 2...
  Llamada API 3: Origenes batch 1, Destinos batch 3...
  Llamada API 4: Origenes batch 1, Destinos batch 4...
  Llamada API 5: Origenes batch 1, Destinos batch 5...
  Llamada API 6: Origenes batch 1, Destinos batch 6...
  Llamada API 7: Origenes batch 1, Destinos batch 7...
  Llamada API 8: Origenes batch 2, Destinos batch 1...
  Llamada API 9: Origenes batch 2, Destinos batch 2...
  Llamada API 10: Origenes batch 2, Destinos batch 3...
  Llamada API 11: Origenes batch 2, Destinos batch 4...
  Llamada API 12: Origenes batch 2, Destinos batch 5...
  Llamada API 13: Origenes batch 2, Destinos batch 6...
  Llamada API 14: Origenes batch 2, Destinos batch 7...
  Llamada API 15: Origenes batch 3, Destinos b

## Modelo Ver 2 y solver

In [ ]:
# --- Markdown ---
# ## 3. Pyomo Model Definition
#
# Definir la estructura del modelo Pyomo (sets, parámetros, variables, objetivo, restricciones)
# usando los datos cargados de CSV y las distancias calculadas via OSRM.
# --- End Markdown ---

# --- Python Code ---
# Crear un modelo concreto
model = pyo.ConcreteModel(name="LogistiCo_VRP_CSV_OSRM_Batched_Final")

# --- Sets ---
model.I = pyo.Set(initialize=set_I_data, doc="Distribution Centers")
model.J = pyo.Set(initialize=set_J_data, doc="Customers")
model.K = pyo.Set(initialize=set_K_data, doc="Vehicles")
model.N = pyo.Set(initialize=set_N_data, doc="All Nodes")

# --- Parameters ---
model.A = pyo.Param(model.I, initialize=param_A_data, default=float('inf'), doc="Capacity of CD i")
model.D = pyo.Param(model.J, initialize=param_D_data, doc="Demand of customer j")
model.Q = pyo.Param(model.K, initialize=param_Q_data, doc="Capacity of vehicle k")
model.R = pyo.Param(model.K, initialize=param_R_data, doc="Range of vehicle k")
model.d = pyo.Param(model.N, model.N, initialize=param_d_data, default=float('inf'), doc="Distance from u to v (km)")
model.c = pyo.Param(model.N, model.N, initialize=param_c_data, default=float('inf'), doc="Cost from u to v (COP)")
model.n_cust = pyo.Param(initialize=param_n_cust_data, doc="Number of customers")

# --- Decision Variables ---
model.x = pyo.Var(model.N, model.N, model.K, within=pyo.Binary, doc="Vehicle k travels from u to v")
model.y = pyo.Var(model.I, model.K, within=pyo.Binary, doc="Vehicle k assigned to CD i")
model.u = pyo.Var(model.J, model.K, within=pyo.NonNegativeReals, bounds=(0, model.n_cust + 1), doc="MTZ auxiliary variable") # Ajuste ligero de bound superior

# --- Objective Function ---
def objective_rule(mod):
    # Penalizar arcos con costo infinito para evitar su uso si es posible
    cost = sum(mod.c[u, v] * mod.x[u, v, k]
               for k in mod.K for u in mod.N for v in mod.N
               if u != v and mod.c[u, v] != float('inf'))
    penalty = sum(1e12 * mod.x[u, v, k] # Penalidad muy grande
                   for k in mod.K for u in mod.N for v in mod.N
                   if u != v and mod.c[u, v] == float('inf'))
    return cost + penalty
model.objective = pyo.Objective(rule=objective_rule, sense=pyo.minimize, doc="Minimize total cost")

# --- Constraints ---
# Constraint 1: Customer Coverage
def customer_coverage_rule(mod, j):
    return sum(mod.x[u, j, k] for k in mod.K for u in mod.N if u != j) == 1
model.customer_coverage = pyo.Constraint(model.J, rule=customer_coverage_rule, doc="Each customer visited exactly once")

# Constraint 2a: Flow Conservation at Customer Nodes
def flow_conservation_customer_rule(mod, j, k):
    in_flow = sum(mod.x[u, j, k] for u in mod.N if u != j)
    out_flow = sum(mod.x[j, v, k] for v in mod.N if v != j)
    return in_flow == out_flow
model.flow_conservation_customer = pyo.Constraint(model.J, model.K, rule=flow_conservation_customer_rule, doc="Flow conservation at customer nodes")

# Constraint 2b: Vehicle Departure from Assigned Depot
def vehicle_departure_rule(mod, i, k):
    # Un vehículo solo sale del depósito si está asignado a él
    return sum(mod.x[i, v, k] for v in mod.N if v != i) == mod.y[i, k]
model.vehicle_departure = pyo.Constraint(model.I, model.K, rule=vehicle_departure_rule, doc="Vehicle leaves assigned depot")

# Constraint 2c: Vehicle Return to Assigned Depot
def vehicle_return_rule(mod, i, k):
    # Un vehículo solo regresa al depósito si salió de él
    return sum(mod.x[u, i, k] for u in mod.N if u != i) == mod.y[i, k]
model.vehicle_return = pyo.Constraint(model.I, model.K, rule=vehicle_return_rule, doc="Vehicle returns to assigned depot")

# Constraint 2d: Vehicle Assignment Constraint
def vehicle_assignment_rule(mod, k):
    # Cada vehículo puede ser asignado a máximo un depósito
    return sum(mod.y[i, k] for i in mod.I) <= 1
model.vehicle_assignment = pyo.Constraint(model.K, rule=vehicle_assignment_rule, doc="Each vehicle assigned to at most one depot")

# Constraint 3: Vehicle Capacity
def vehicle_capacity_rule(mod, k):
    # Suma la demanda D_j solo si el cliente j es visitado por el vehículo k
    demand_served_by_k = sum(mod.D[j] * sum(mod.x[u, j, k] for u in mod.N if u != j) for j in mod.J)
    return demand_served_by_k <= mod.Q[k]
model.vehicle_capacity = pyo.Constraint(model.K, rule=vehicle_capacity_rule, doc="Vehicle capacity constraint")

# Constraint 4: Vehicle Range (Autonomy)
# Usamos un conjunto indexado para la restricción de rango
model.RangeConstraintSet = pyo.Set(initialize=model.K)
def vehicle_range_rule(mod, k):
    # Suma la distancia d_uv solo si el arco (u,v) es usado por el vehículo k y no es infinito
    dist_traveled = sum(mod.d[u, v] * mod.x[u, v, k]
                       for u in mod.N for v in mod.N
                       if u != v and mod.d[u,v] != float('inf'))
    return dist_traveled <= mod.R[k]
model.vehicle_range = pyo.Constraint(model.RangeConstraintSet, rule=vehicle_range_rule, doc="Vehicle range constraint")

# Constraint 4b: Ensure no infeasible arcs are used (complementary to objective penalty)
# Crear un conjunto de arcos inviables (u, v, k)
def infeasible_arcs_indices(mod):
    for u in mod.N:
        for v in mod.N:
            if u != v and mod.d[u,v] == float('inf'):
                for k in mod.K:
                     yield (u, v, k)
model.InfeasibleArcsSet = pyo.Set(initialize=infeasible_arcs_indices, dimen=3)

def forbid_infeasible_arcs_rule(mod, u, v, k):
    return mod.x[u,v,k] == 0
model.ForbidInfeasibleArcs = pyo.Constraint(model.InfeasibleArcsSet, rule=forbid_infeasible_arcs_rule, doc="Forbid using arcs with infinite distance")


# Constraint 5: Distribution Center Capacity (Omitted)
print("Skipping CD Capacity Constraint (5).")

# Constraint 6: Subtour Elimination (MTZ)
# Se aplica SOLO entre pares de nodos de CLIENTES distintos j y j'
def subtour_elimination_rule(mod, j, j_prime, k):
    if j == j_prime or j not in mod.J or j_prime not in mod.J: # Asegurar que ambos son clientes
        return pyo.Constraint.Skip
    # No necesitamos chequear d[j,j_prime] aquí si ya forzamos x=0 para arcos infinitos
    return mod.u[j, k] - mod.u[j_prime, k] + mod.n_cust * mod.x[j, j_prime, k] <= mod.n_cust - 1
model.subtour_elimination = pyo.Constraint(model.J, model.J, model.K, rule=subtour_elimination_rule, doc="MTZ subtour elimination")

# Bounds para las variables auxiliares u_jk (importante para MTZ)
def u_bounds_rule(mod, j, k):
     return (0, mod.u[j, k], mod.n_cust)
model.u_bounds = pyo.Constraint(model.J, model.K, rule=u_bounds_rule, doc="Bounds for MTZ auxiliary variables")

# Constraint 7: Prevent trivial loops (x_uuk = 0)
# Usar un índice combinado N x N x K para x
model.x_index = pyo.Set(initialize = [(u,v,k) for u in model.N for v in model.N for k in model.K])
def no_trivial_loops_rule(mod, u, k):
    if (u, u, k) in model.x_index: # Chequear si el índice existe para la variable
         return mod.x[u, u, k] == 0
    else:
         return pyo.Constraint.Skip # No debería pasar con variables densas
model.no_trivial_loops = pyo.Constraint(model.N, model.K, rule=no_trivial_loops_rule, doc="Prevent travel from a node to itself")

print("Pyomo model structure defined using data from CSVs and OSRM API (Batched Corrected Func).")
# --- End Python Code ---

In [ ]:
# --- Markdown ---
# ## 4. Solve the Model and Display Results
#
# Use a selected MILP solver (e.g., GLPK, CBC) to find the optimal solution
# for the VRP model and display the results, including cost, assignments, and routes.
# --- End Markdown ---

# --- Python Code ---
solver_name = 'gurobi' # Or 'cbc'
try:
    solver = pyo.SolverFactory(solver_name)
    if not solver.available():
        raise RuntimeError(f"Solver '{solver_name}' not found or not executable.")
except Exception as e:
    print(f"ERROR: Could not find or initialize solver '{solver_name}'. Please install it and ensure it's in your PATH. Error: {e}", file=sys.stderr)
    print("You can try installing GLPK or CBC (e.g., using conda).", file=sys.stderr)
    sys.exit(1)


print(f"\nSolving the model using {solver_name}...")
results = solver.solve(model, tee=True) # tee=True muestra la salida del solver

# --- Display Results ---

print("\n--- Solver Results ---")
print(results) # Imprime el objeto detallado de resultados del solver

# Verificar estado de la solución de forma más robusta
solution_found = False
solution_optimal = False
if results.solver.status == pyo.SolverStatus.ok or results.solver.status == pyo.SolverStatus.warning:
    if results.solver.termination_condition in [pyo.TerminationCondition.optimal,
                                               pyo.TerminationCondition.feasible,
                                               pyo.TerminationCondition.maxTimeLimit,
                                               pyo.TerminationCondition.minFunctionValue, # Algunos solvers usan esto
                                               pyo.TerminationCondition.minStepLength,    # Otros para factibilidad
                                               pyo.TerminationCondition.globallyOptimal, # Para solvers globales
                                               pyo.TerminationCondition.locallyOptimal   # Para solvers NLP
                                               ]:
        if hasattr(results, "solution") and len(results.solution) > 0 and results.solution(0).status != pyo.SolutionStatus.unknown :
             try:
                 model.solutions.load_from(results)
                 solution_found = True
                 if results.solver.termination_condition == pyo.TerminationCondition.optimal:
                     solution_optimal = True
             except (AttributeError, ValueError, KeyError) as e:
                  print(f"\n--- Advertencia: El solver indicó éxito, pero ocurrió un error al cargar la solución: {e} ---")
        else:
             print("\n--- El solver indicó éxito, pero no se cargaron datos de solución en el objeto de resultados o el estado es desconocido. ---")
    else:
        print(f"\n--- El solver terminó con estado {results.solver.status} pero la condición de terminación es {results.solver.termination_condition}. ---")
else:
     print(f"\n--- El solver falló o no encontró una solución. Estado: {results.solver.status}, Condición: {results.solver.termination_condition} ---")


if solution_found:
    if solution_optimal:
        print("\n--- Solución Óptima Encontrada ---")
    elif results.solver.termination_condition == pyo.TerminationCondition.maxTimeLimit:
         print("\n--- Límite de Tiempo Alcanzado - Solución Factible Encontrada (puede no ser óptima) ---")
    else:
        print(f"\n--- Solución Factible Encontrada (Condición de terminación: {results.solver.termination_condition}) ---")

    try:
        objective_value = pyo.value(model.objective)
        print(f"Costo Total Mínimo: {objective_value:,.2f} COP")
    except Exception as e:
        print(f"No se pudo obtener el valor del objetivo: {e}")


    print("\n--- Asignaciones de Vehículos (y_ik) ---")
    assigned_vehicles_count = 0
    assignments = {}
    for k in model.K:
        assignments[k] = 'Unassigned'
        for i in model.I:
            y_val = pyo.value(model.y[i, k], exception=False)
            if y_val is not None and y_val > 0.5:
                print(f"Vehículo {k} inicia desde CD {i}")
                assignments[k] = i
                assigned_vehicles_count += 1
                break
        if assignments[k] == 'Unassigned':
             print(f"Vehículo {k} no se utiliza.")
    if assigned_vehicles_count == 0:
        print("No se asignó ningún vehículo a ninguna ruta.")

    print("\n--- Rutas (x_uvk) ---")
    total_distance = 0
    total_demand_served = 0
    vehicles_used = set()

    for k in model.K:
        start_depot = assignments.get(k)
        if start_depot != 'Unassigned':
            print(f"\nRuta para Vehículo {k} (desde {start_depot}):")
            route = [start_depot]
            current_node = start_depot
            route_distance = 0
            route_demand = 0
            visited_nodes_in_route = {start_depot} # Track visited nodes for this specific route
            route_found = False
            route_error = None # Store specific error message
            max_steps = len(model.N) + 2

            for step in range(max_steps):
                next_node_found = False
                the_next_node = None
                possible_next_count = 0

                # Find the next node
                for v in model.N:
                    if current_node != v:
                        x_val = pyo.value(model.x[current_node, v, k], exception=False)
                        if x_val is not None and x_val > 0.5:
                             if abs(x_val - 1.0) < 1e-6:
                                 the_next_node = v
                                 next_node_found = True
                                 possible_next_count += 1
                                 if possible_next_count > 1:
                                     route_error = f"Error: Multiple next steps ({possible_next_count}) found from {current_node}"
                                     break

                if route_error: break # Exit inner loop

                # Check for issues after iterating through potential next nodes
                if not next_node_found:
                    if current_node != start_depot:
                        route_error = f"Error: No next step found from {current_node}. Incomplete route."
                    # else: Vehicle assigned but didn't move (handled after loop)
                    break # Exit inner loop

                # Process the found next node
                next_node = the_next_node
                arc_dist = model.d[current_node, next_node]
                if arc_dist == float('inf'):
                    route_error = f"Error: Route uses an infeasible arc from {current_node} to {next_node}"
                    route.append(f"{next_node}(inf)")
                    break
                route_distance += arc_dist
                route.append(next_node)

                if next_node == start_depot: # Completed the loop
                    route_found = True
                    break

                if next_node in visited_nodes_in_route:
                    route_error = f"Error: Cycle detected, node {next_node} visited again"
                    route.append("CycleErr")
                    break
                visited_nodes_in_route.add(next_node)

                if next_node in model.J:
                    route_demand += model.D[next_node]

                current_node = next_node # Move to the next node

                # Safety break
                if step == max_steps - 1:
                    route_error = f"Error: Route did not complete within {max_steps} steps"
                    route.append("...TimeoutErr")
                    break

            # Print route details based on findings
            if route_found and not route_error:
                 print(f"  {' -> '.join(route)}")
                 print(f"  Distance: {route_distance:.2f} km (Max: {model.R[k]:.2f})")
                 print(f"  Demand: {route_demand:.2f} kg (Capacity: {model.Q[k]:.2f})")
                 total_distance += route_distance
                 total_demand_served += route_demand
                 vehicles_used.add(k)
                 # Validation checks
                 if route_distance > model.R[k] + 1e-6: print(f"  ADVERTENCIA: Vehículo {k} excedió rango!")
                 if route_demand > model.Q[k] + 1e-6: print(f"  ADVERTENCIA: Vehículo {k} excedió capacidad!")
            elif route_error:
                 print(f"  Problema en Ruta: {' -> '.join(route)} ({route_error})")
            elif len(route) == 1 and assignments.get(k) != 'Unassigned':
                 # This case means y[i,k]=1 but no x[i,v,k]=1 was found
                 vehicle_moves = any(pyo.value(model.x[u, v, k], exception=False) > 0.5 for u in model.N for v in model.N if u != v)
                 if not vehicle_moves:
                      print(f"  Vehículo {k} asignado a {start_depot} pero no realizó ninguna ruta.")
                 else: # Should have been caught as an error before
                      print(f"  Ruta extraña para Vehículo {k}: {route}")


    print("\n--- Resumen General ---")
    print(f"Distancia total cubierta por vehículos asignados: {total_distance:.2f} km")
    print(f"Demanda total servida por vehículos asignados: {total_demand_served:.2f} kg")
    print(f"Demanda total esperada de clientes: {sum(param_D_data.values()):.2f} kg")
    print(f"Número de vehículos utilizados: {len(vehicles_used)} de {len(model.K)}")
    if abs(total_demand_served - sum(param_D_data.values())) > 1e-6:
        print("ADVERTENCIA: ¡La demanda total servida no coincide con la demanda total esperada! Verificar infactibilidad o restricciones del modelo.")


elif results.solver.termination_condition == pyo.TerminationCondition.infeasible:
     print("\n--- Modelo Infactible ---")
     print("El solver determinó que no existe solución que satisfaga todas las restricciones.")
     print("Revisa las restricciones (capacidades, rangos) y los datos (e.j., nodos desconectados por errores OSRM, demanda excediendo capacidad total).")

else:
    print("\n--- El Solver no encontró una Solución Óptima o Factible ---")
    print(f"Estado del Solver: {results.solver.status}")
    print(f"Condición de Terminación: {results.solver.termination_condition}")
    print("Revisa los logs del solver (salida tee=True) y la formulación del modelo por errores.")

# --- End Python Code ---

# --- Markdown ---
# ## 5. Generación de Archivos y Reportes
# --- End Markdown ---

# --- Python Code ---
import os
import csv

# Asegúrate de que las variables de entorno estén definidas o define valores por defecto
group_name = os.getenv('GROUP_NAME', 'Grupo5') # Nombre por defecto
case_type = os.getenv('CASE_TYPE', 'estandar') # Tipo por defecto
case_number = os.getenv('CASE_NUMBER', '3') # Número/Identificador por defecto

def generate_reports(solution_model, data_dict):
    """
    Genera los archivos de rutas, valor objetivo y costos operacionales.
    Usa el objeto 'model' de Pyomo que ya tiene la solución cargada.
    """
    group_name = os.getenv('GROUP_NAME', 'GrupoX')
    case_type = os.getenv('CASE_TYPE', 'estandar')
    case_number = os.getenv('CASE_NUMBER', 'Test')

    routes_filename = f"{group_name}-caso-{case_type}-{case_number}-ruta.csv"
    objective_filename = f"Caso{case_number}_Objetivo.txt"
    operational_cost_filename = f"InformeCostosOperacionales{case_number}.txt"

    # --- 2. Archivo de Rutas (.csv) ---
    routes_entries = []
    # Reconstruir asignaciones y rutas directamente desde el modelo Pyomo
    assignments = {}
    for k in solution_model.K:
        assignments[k] = 'Unassigned'
        for i in solution_model.I:
            y_val = pyo.value(solution_model.y[i, k], exception=False)
            if y_val is not None and y_val > 0.5:
                assignments[k] = i
                break

    for k in solution_model.K:
        start_depot = assignments.get(k)
        if start_depot != 'Unassigned':
            current_node = start_depot
            visited_in_route = {start_depot}
            max_steps = len(solution_model.N) + 2
            route_complete = False
            error_in_route = False

            for step in range(max_steps):
                next_node = None
                found_next = False
                possible_next_count = 0
                for v in solution_model.N:
                    if current_node != v:
                        x_val = pyo.value(solution_model.x[current_node, v, k], exception=False)
                        if x_val is not None and x_val > 0.5:
                           if abs(x_val - 1.0) < 1e-6:
                                next_node = v
                                found_next = True
                                possible_next_count +=1
                                if possible_next_count > 1:
                                     print(f"Advertencia Reporte: Múltiples salidas para Vehículo {k} desde {current_node}.")
                                     error_in_route = True; break

                if error_in_route: break

                if not found_next:
                    if current_node == start_depot: # Vehicle didn't leave or returned immediately
                       # Check if it ever left
                       if any(pyo.value(solution_model.x[start_depot, v, k], exception=False) > 0.5 for v in solution_model.N if v != start_depot):
                           # It left but didn't find a next node (should not happen in feasible solution)
                           print(f"Advertencia Reporte: Ruta para Vehículo {k} parece terminar prematuramente en {current_node}.")
                       # Else: Vehicle was assigned but not used in route (y=1, but all x=0), fine.
                    else:
                        print(f"Advertencia Reporte: Ruta incompleta para Vehículo {k} (termina en {current_node}).")
                    break # End route reconstruction here

                # Add step to entries
                routes_entries.append({
                    'ID-Vehiculo': k,
                    'ID-Origen': current_node,
                    'ID-Destino': next_node
                })

                if next_node == start_depot:
                    route_complete = True
                    break # Finished route

                if next_node in visited_in_route:
                    print(f"Advertencia Reporte: Ciclo detectado en ruta de Vehículo {k} (nodo {next_node}).")
                    error_in_route = True; break
                visited_in_route.add(next_node)
                current_node = next_node

                if step == max_steps - 1:
                     print(f"Advertencia Reporte: Ruta para Vehículo {k} excedió max_steps.")
                     error_in_route = True; break


    # Escribir archivo CSV
    try:
        with open(routes_filename, mode='w', newline='', encoding='utf-8') as csvfile:
            fieldnames = ['ID-Vehiculo', 'ID-Origen', 'ID-Destino']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(routes_entries)
    except IOError as e:
        print(f"Error escribiendo archivo de rutas {routes_filename}: {e}", file=sys.stderr)

    # --- 3. Archivo de Valor Objetivo (.txt) ---
    try:
        total_cost = pyo.value(solution_model.objective)
        cost_str = f"{total_cost:.4f}" # Usar más decimales para precisión
    except Exception as e:
        print(f"Advertencia: No se pudo obtener 'TotalCost' del modelo para el reporte: {e}", file=sys.stderr)
        cost_str = "No disponible"

    try:
        with open(objective_filename, mode='w', encoding='utf-8') as obj_file:
            obj_file.write(f"{cost_str}\n")
    except IOError as e:
         print(f"Error escribiendo archivo objetivo {objective_filename}: {e}", file=sys.stderr)

    # --- 4. Informe de Costos Operacionales (.txt) ---
    # Estos costos auxiliares NO están definidos en el modelo actual.
    # Necesitarías añadirlos como variables y restricciones si quieres reportarlos.
    cost_components_to_report = {
        # 'CostCarTot': "Costo Carga Total", # No en el modelo
        # 'CostDistTot': "Costo Distancia Total", # No en el modelo
        # 'CostTpTot': "Costo Tarifa Horaria Total", # No en el modelo
        # 'CostRecTot': "Costo Recarga Total", # No en el modelo
        # 'CostMant': "Costo Mantenimiento" # No en el modelo
    }

    try:
        with open(operational_cost_filename, mode='w', encoding='utf-8') as cost_file:
            cost_file.write("Informe de Costos Operacionales\n")
            cost_file.write("================================\n\n")
            cost_file.write(f"Costo Total (Función Objetivo): {cost_str}\n\n")
            cost_file.write("Desglose de costos no disponible en este modelo.\n") # Indicar que no están
            # (Si añades las variables de costo auxiliar, descomenta y adapta el bucle)
            # for comp_var, comp_name in cost_components_to_report.items():
            #     cost_value = pyo.value(getattr(solution_model, comp_var, None), exception=False)
            #     value_str = f"{cost_value:.2f}" if isinstance(cost_value, (int, float)) else "No disponible"
            #     cost_file.write(f"{comp_name}: {value_str}\n")

    except IOError as e:
         print(f"Error escribiendo archivo de costos operacionales {operational_cost_filename}: {e}", file=sys.stderr)

    print(f"\nArchivos de reporte generados:")
    print(f"- {routes_filename}")
    print(f"- {objective_filename}")
    print(f"- {operational_cost_filename}")


# Llamar a la función de generación de reportes si se encontró una solución
if solution_found:
     generate_reports(model, data) # Pasar el objeto 'model' de Pyomo y el diccionario 'data' original
else:
     print("\nNo se encontró solución, no se generarán reportes.")
# --- End Python Code ---

In [ ]:
# --- Markdown ---
# ## 6. Visualización con Mapas
# --- End Markdown ---

# --- Python Code ---
# Import necessary libraries
try:
    import folium
    import matplotlib.pyplot as plt
    import matplotlib.colors as mcolors
    import numpy as np
    from branca.element import Template, MacroElement
    folium_installed = True
except ImportError:
    print("Folium no está instalado. Para visualizar el mapa, por favor instálalo: pip install folium matplotlib")
    folium_installed = False


if folium_installed and solution_found: # Solo intentar si folium está y hay solución

    # Function to extract routes from the solution model for map
    def extract_routes_for_map(solution_model, data_dict):
        """Extrae rutas del modelo Pyomo resuelto para el mapa."""
        routes = {} # {vehicle_id: [node_id, node_id,...]}
        assignments = {}
        for k in solution_model.K:
            assignments[k] = 'Unassigned'
            for i in solution_model.I:
                y_val = pyo.value(solution_model.y[i, k], exception=False)
                if y_val is not None and y_val > 0.5:
                    assignments[k] = i
                    break

        for v in solution_model.K:
            start_depot = assignments.get(v)
            if start_depot == 'Unassigned':
                routes[v] = []
                continue

            route = [start_depot]
            current_node = start_depot
            visited_in_route = {start_depot}
            max_steps = len(solution_model.N) + 2
            route_complete = False
            error_in_route = False

            for step in range(max_steps):
                next_node = None
                found_next = False
                possible_next_count = 0
                for j in solution_model.N:
                    if current_node != j:
                        x_val = pyo.value(solution_model.x[current_node, j, v], exception=False)
                        if x_val is not None and x_val > 0.5:
                           if abs(x_val - 1.0) < 1e-6:
                                next_node = j
                                found_next = True
                                possible_next_count +=1
                                if possible_next_count > 1: error_in_route = True; break
                if error_in_route: break

                if not found_next:
                    # Check if returning to depot
                    x_val_return = pyo.value(solution_model.x[current_node, start_depot, v], exception=False)
                    if x_val_return is not None and x_val_return > 0.5:
                        route.append(start_depot)
                        route_complete = True
                    else:
                         if current_node != start_depot: # Error only if not at depot
                              print(f"Advertencia Mapeo: Ruta incompleta para Vehículo {v} (termina en {current_node}).")
                              error_in_route = True
                    break

                route.append(next_node)

                if next_node == start_depot:
                    route_complete = True
                    break

                if next_node in visited_in_route:
                    print(f"Advertencia Mapeo: Ciclo detectado en ruta de Vehículo {v} (nodo {next_node}).")
                    error_in_route = True; break
                visited_in_route.add(next_node)
                current_node = next_node

                if step == max_steps - 1:
                     print(f"Advertencia Mapeo: Ruta para Vehículo {v} excedió max_steps.")
                     error_in_route = True; break

            if route_complete and not error_in_route:
                routes[v] = route
            else:
                print(f"Advertencia Mapeo: Ruta incompleta o con error para Vehículo {v}. Ruta parcial: {route}")
                routes[v] = [] # Marcar como vacía

        return routes

    # Function to create map (using data_dict)
    def create_map(data_dict):
        # Use the original data dictionary passed to generate_reports
        coords_dict = data_dict['UbicacionNodo']
        if not coords_dict: return folium.Map(location=[4.6097, -74.0817], zoom_start=11)
        lats = [loc[0] for loc in coords_dict.values()]
        longs = [loc[1] for loc in coords_dict.values()]
        if not lats or not longs: return folium.Map(location=[4.6097, -74.0817], zoom_start=11)
        avg_lat = sum(lats) / len(lats)
        avg_long = sum(longs) / len(longs)
        return folium.Map(location=[avg_lat, avg_long], zoom_start=12)

    # Function to add legend (same as before)
    def add_legend(m, vehicle_colors):
        items = ''.join(
            f'<i style="background:{color};width:10px;height:10px;float:left;margin-right:5px;"></i>{vehicle}<br>'
            for vehicle, color in vehicle_colors.items() if color
        )
        if not items: return
        legend_html = f'''
        <div style="
            position: fixed; bottom: 50px; left: 50px; width: 160px; height: auto; z-index:9999;
            font-size:14px; background-color:white; padding:10px; border:2px solid grey; border-radius:5px;">
            <b>Rutas de Vehículos</b><br>
            {items}
        </div>
        '''
        legend = MacroElement()
        legend._template = Template(legend_html)
        m.get_root().add_child(legend)

    # Function to plot routes and markers (using solution_model and data_dict)
    def plot_routes_on_map(m, routes, data_dict, solution_model):
        active_routes = {v: r for v, r in routes.items() if r and len(r) > 1}
        num_vehicles = len(active_routes)
        if num_vehicles == 0:
            print("No hay rutas activas para visualizar en el mapa.")
            # Add markers even if no routes
            for node_id, coords in data_dict['UbicacionNodo'].items():
                 lat, lon = coords
                 node_type = data_dict['TipoNodo'][node_id]
                 icon = folium.Icon(
                     color='blue' if node_type == 'Depósito' else 'green',
                     icon='home' if node_type == 'Depósito' else 'user',
                     prefix='fa'
                 )
                 folium.Marker(location=(lat, lon), icon=icon, tooltip=f"Nodo {node_id} ({node_type})").add_to(m)
            return

        cmap = plt.cm.get_cmap('hsv', num_vehicles)
        vehicle_colors = {}
        active_vehicle_ids = sorted(active_routes.keys())
        for idx, v in enumerate(active_vehicle_ids):
             vehicle_colors[v] = mcolors.rgb2hex(cmap(idx))

        for v, route in active_routes.items():
            color = vehicle_colors[v]
            route_coords = [data_dict['UbicacionNodo'][node_id] for node_id in route if node_id in data_dict['UbicacionNodo']]
            if len(route_coords) < 2: continue
            folium.PolyLine(locations=route_coords, color=color, weight=3, opacity=0.7).add_to(m)

        # Add markers for all nodes
        for node_id in data_dict['N']:
            coords = data_dict['UbicacionNodo'].get(node_id)
            if not coords: continue
            lat, lon = coords
            node_type = data_dict['TipoNodo'][node_id]

            vehicle_info = "No Visitado"
            marker_color = 'gray'
            icon_type = 'question-circle'

            if node_type == 'Depósito':
                icon_type = 'home'
                marker_color = 'blue'
                vehicles_at_depot = [k for k in solution_model.K if assignments.get(k) == node_id] # Use assignments from report func scope
                if vehicles_at_depot:
                    vehicle_info = "Inicia/Termina: " + ", ".join(vehicles_at_depot)
                else:
                     vehicle_info = "No utilizado"
            else: # Client node
                icon_type = 'user'
                marker_color = 'green'
                vehicle_visiting = None
                # Check which vehicle visits this client node j
                for k in active_vehicle_ids:
                    if sum(pyo.value(solution_model.x[u, node_id, k], exception=False) or 0 for u in solution_model.N if u != node_id) > 0.5:
                         vehicle_visiting = k
                         break
                if vehicle_visiting:
                    marker_color = vehicle_colors.get(vehicle_visiting, 'green')
                    vehicle_info = f"Servido por {vehicle_visiting}"
                else:
                     marker_color = 'red'
                     vehicle_info = "NO SERVIDO"

            popup_text = f"<b>Nodo:</b> {node_id}<br><b>Tipo:</b> {node_type}<br><b>{vehicle_info}</b>"
            if node_type == 'Cliente':
                 popup_text += f"<br><b>Demanda:</b> {data_dict['DemandaEnvio'][node_id]}"

            icon = folium.Icon(color=marker_color, icon=icon_type, prefix='fa')
            folium.Marker(
                location=(lat, lon),
                icon=icon,
                popup=folium.Popup(popup_text, max_width=300),
                tooltip=f"Nodo {node_id} ({node_type})"
            ).add_to(m)

        add_legend(m, vehicle_colors)

    # --- Generate and Display Map ---
    routes_map = extract_routes_for_map(model, data) # Use the solved Pyomo model 'model' and original 'data'
    map_viz = create_map(data) # Use original 'data'
    plot_routes_on_map(map_viz, routes_map, data, model) # Pass 'model'

    display(map_viz) # Display in Jupyter/compatible environment

    map_filename = f"{group_name}-caso-{case_type}-{case_number}-mapa.html"
    map_viz.save(map_filename)
    print(f"\nMapa guardado en '{map_filename}'")

else:
    print("\nNo se encontró solución, no se puede generar el mapa.")

# --- End Python Code ---